In [18]:
!pip install azure-ai-ml


In [1]:
from azure.ai.ml.entities import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core import Workspace
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

ENVIRONMENT_PATH = os.getenv("ENVIRONMENT_PATH") # Ruta al archivo environment.yml
MODEL_NAME = os.getenv("MODEL_NAME")

credential = AzureCliCredential()

# Cargo el workspace configurado
workspace = Workspace.from_config() 

# Cargo el archivo environment.yml
env = Environment(
    name="UKEnergyModelEnvironment",
    description="Entorno de ML para predicciones de energía",
    version="1",
    conda_file=ENVIRONMENT_PATH,  # Archivo con dependencias Conda
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",  # Imagen base de Docker
)

registered_model = Model(workspace, name=MODEL_NAME)
print(f"Nombre: {registered_model.name}, Versión: {registered_model.version}")

Nombre: tensorflow_series_UK_energy, Versión: 4


In [ ]:
access_token = credential.get_token("https://management.azure.com/.default")
with open("token.txt","w+") as tokenFile:
    tokenFile.write("TOKEN = "+access_token.token)

In [10]:
scripts_dir = "./"

# Configuración del despliegue
model_deployment = ManagedOnlineDeployment(
    name="UKEnergyModelDeploy",
    endpoint_name="uk-energy-model-endpoint",
    description="Endpoint de inferencia del modelo de predicción de energia de UK",
    model=registered_model,  # Modelo registrado en el workspace
    environment=env,  # Entorno configurado previamente
    code_configuration=CodeConfiguration(
        code=os.path.join(scripts_dir, "scripts"),  # Directorio de scripts
        scoring_script="inference_score.py"  # Script de inferencia
    ),
    instance_type="Standard_DS4_v2",
    instance_count=1,
    environment_variables={
        "MODEL_PATH": "azureml-models/tensorflow_series_model/1/",
        "SCALER_PATH": "azureml-models/tensorflow_series_model/scaler.pkl",
        "TARGET_INDEX": "0",
        "WINDOW_SIZE": "48"
    }
)

# Implementar el despliegue
ml_client = MLClient.from_config(credential)
ml_client.online_deployments.get_logs(
    name="tensorflow-series-uk-energy-3", endpoint_name="uk-energy-model-endpoint", lines=1000
)
#ml_client.environments.create_or_update(env)
#ml_client.begin_create_or_update(model_deployment).wait()


Found the config file in: /config.json


'Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: InProgress\n\nContainer events:\nKind: Pod, Name: Pulling, Type: Normal, Time: 2024-12-06T19:49:01.170064Z, Message: Start pulling container image\nKind: Pod, Name: Downloading, Type: Normal, Time: 2024-12-06T19:49:01.617457Z, Message: Start downloading models\nKind: Pod, Name: Pulled, Type: Normal, Time: 2024-12-06T19:49:40.784018Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2024-12-06T19:49:40.784018Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2024-12-06T19:49:40.961159Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2024-12-06T19:49:41.037167Z, Message: Started container inference-server\n\nContainer logs:\n2024-12-06T19:49:41,047232255+00:00 - gunicorn/run \n2024-12-06T19:49:41,049214667+00:00 - nginx/run \n2024-1